In [1]:
# Mount Google Drive
from google.colab import drive # import drive from google colab
 
ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)
 
drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Mounted at /content/drive


In [1]:
!pip install pennylane
from IPython.display import clear_output
clear_output()

In [ ]:
import os

def restart_runtime():
  os.kill(os.getpid(), 9)
restart_runtime()

In [1]:
# %matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import numpy as np

import tensorflow as tf
from tensorflow.keras.utils import to_categorical

# Loading Raw Data

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train[:, 0:27, 0:27]
x_test = x_test[:, 0:27, 0:27]

In [3]:
x_train_flatten = x_train.reshape(x_train.shape[0], x_train.shape[1]*x_train.shape[2])/255.0
x_test_flatten = x_test.reshape(x_test.shape[0], x_test.shape[1]*x_test.shape[2])/255.0

In [4]:
print(x_train_flatten.shape, y_train.shape)
print(x_test_flatten.shape, y_test.shape)

(60000, 729) (60000,)
(10000, 729) (10000,)


In [5]:
x_train_0 = x_train_flatten[y_train == 0]
x_train_1 = x_train_flatten[y_train == 1]
x_train_2 = x_train_flatten[y_train == 2]
x_train_3 = x_train_flatten[y_train == 3]
x_train_4 = x_train_flatten[y_train == 4]
x_train_5 = x_train_flatten[y_train == 5]
x_train_6 = x_train_flatten[y_train == 6]
x_train_7 = x_train_flatten[y_train == 7]
x_train_8 = x_train_flatten[y_train == 8]
x_train_9 = x_train_flatten[y_train == 9]

x_train_list = [x_train_0, x_train_1, x_train_2, x_train_3, x_train_4, x_train_5, x_train_6, x_train_7, x_train_8, x_train_9]

print(x_train_0.shape)
print(x_train_1.shape)
print(x_train_2.shape)
print(x_train_3.shape)
print(x_train_4.shape)
print(x_train_5.shape)
print(x_train_6.shape)
print(x_train_7.shape)
print(x_train_8.shape)
print(x_train_9.shape)

(5923, 729)
(6742, 729)
(5958, 729)
(6131, 729)
(5842, 729)
(5421, 729)
(5918, 729)
(6265, 729)
(5851, 729)
(5949, 729)


In [6]:
x_test_0 = x_test_flatten[y_test == 0]
x_test_1 = x_test_flatten[y_test == 1]
x_test_2 = x_test_flatten[y_test == 2]
x_test_3 = x_test_flatten[y_test == 3]
x_test_4 = x_test_flatten[y_test == 4]
x_test_5 = x_test_flatten[y_test == 5]
x_test_6 = x_test_flatten[y_test == 6]
x_test_7 = x_test_flatten[y_test == 7]
x_test_8 = x_test_flatten[y_test == 8]
x_test_9 = x_test_flatten[y_test == 9]

x_test_list = [x_test_0, x_test_1, x_test_2, x_test_3, x_test_4, x_test_5, x_test_6, x_test_7, x_test_8, x_test_9]

print(x_test_0.shape)
print(x_test_1.shape)
print(x_test_2.shape)
print(x_test_3.shape)
print(x_test_4.shape)
print(x_test_5.shape)
print(x_test_6.shape)
print(x_test_7.shape)
print(x_test_8.shape)
print(x_test_9.shape)

(980, 729)
(1135, 729)
(1032, 729)
(1010, 729)
(982, 729)
(892, 729)
(958, 729)
(1028, 729)
(974, 729)
(1009, 729)


# Selecting the dataset

Output: X_train, Y_train, X_test, Y_test

In [251]:
n_train_sample_per_class = 200
n_class = 4

X_train = x_train_list[0][:n_train_sample_per_class, :]
Y_train = np.zeros((X_train.shape[0]*n_class,), dtype=int)

for i in range(n_class-1):
    X_train = np.concatenate((X_train, x_train_list[i+1][:n_train_sample_per_class, :]), axis=0)
    Y_train[(i+1)*n_train_sample_per_class:(i+2)*n_train_sample_per_class] = i+1

X_train.shape, Y_train.shape

((800, 729), (800,))

In [252]:
n_test_sample_per_class = int(0.25*n_train_sample_per_class)

X_test = x_test_list[0][:n_test_sample_per_class, :]
Y_test = np.zeros((X_test.shape[0]*n_class,), dtype=int)

for i in range(n_class-1):
    X_test = np.concatenate((X_test, x_test_list[i+1][:n_test_sample_per_class, :]), axis=0)
    Y_test[(i+1)*n_test_sample_per_class:(i+2)*n_test_sample_per_class] = i+1

X_test.shape, Y_test.shape

((200, 729), (200,))

# Dataset Preprocessing

In [253]:
def normalize(X, use_params=False, params=None):
    """Normalize the given dataset X
    Args:
        X: ndarray, dataset
    
    Returns:
        (Xbar, mean, std): tuple of ndarray, Xbar is the normalized dataset
        with mean 0 and standard deviation 1; mean and std are the 
        mean and standard deviation respectively.
    
    Note:
        You will encounter dimensions where the standard deviation is
        zero, for those when you do normalization the normalized data
        will be NaN. Handle this by setting using `std = 1` for those 
        dimensions when doing normalization.
    """
    if use_params:
        mu = params[0]
        std_filled = [1]
    else:
        mu = np.mean(X, axis=0)
        std = np.std(X, axis=0)
        #std_filled = std.copy()
    #std_filled[std==0] = 1.
    Xbar = (X - mu)/(std + 1e-8)
    return Xbar, mu, std


In [254]:
X_train, mu, std = normalize(X_train)
X_test = (X_test - mu)/(std + 1e-8)

X_train.shape, X_test.shape

((800, 729), (200, 729))

In [257]:
X_train = X_train.reshape(X_train.shape[0], 27, 27, 1)
X_test = X_test.reshape(X_test.shape[0], 27, 27, 1)

X_train.shape, X_test.shape

((800, 27, 27, 1), (200, 27, 27, 1))

In [258]:
class_label = np.loadtxt('./tetra_class_label.txt')

In [259]:
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)

# for i in range(n_class):
#     Y_train[Y_train[:, i] == 1.] = class_label[i]

# for i in range(n_class):
#     Y_test[Y_test[:, i] == 1.] = class_label[i]

# Quantum

In [60]:
# session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=16, inter_op_parallelism_threads=16)
# tf.compat.v1.set_random_seed(1)
# sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
# tf.compat.v1.keras.backend.set_session(sess)

In [260]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer, GradientDescentOptimizer

qml.enable_tape()

# Set a random seed
np.random.seed(2020)

In [261]:
# Define output labels as quantum state vectors
def density_matrix(state):
    """Calculates the density matrix representation of a state.

    Args:
        state (array[complex]): array representing a quantum state vector

    Returns:
        dm: (array[complex]): array representing the density matrix
    """
    return np.outer(state,  np.conj(state))

In [262]:
state_labels = np.loadtxt('./tetra_states.txt', dtype=np.complex_)

In [263]:
my_bucket = "amazon-braket-0f5d17943f73"  # the name of the bucket
my_prefix = "Tugas_Akhir"  # the name of the folder in the bucket
s3_folder = (my_bucket, my_prefix)

device_arn = "arn:aws:braket:::device/quantum-simulator/amazon/sv1"

dev_remote = qml.device(
    "braket.aws.qubit",
    device_arn=device_arn,
    wires=9,
    s3_destination_folder=s3_folder,
    parallel=True,
)

In [264]:
n_qubits = n_class

dev = qml.device("default.qubit", wires=n_qubits)

#interface="tf", grad_method="backprop"

@qml.qnode(dev)
def qcircuit(params, inputs):
    """A variational quantum circuit representing the DRC.

    Args:
        params (array[float]): array of parameters
        inputs = [x, y]
        x (array[float]): 1-d input vector
        y (array[float]): single output state density matrix

    Returns:
        float: fidelity between output state and input
    """
    
    # layer iteration
    for l in range(len(params[0])):
        # qubit iteration
        for q in range(n_qubits):
            # gate iteration
            for g in range(int(len(inputs)/3)):
                qml.Rot(*(params[0][l][q][3*g:3*(g+1)] * inputs[3*g:3*(g+1)] + params[1][l][q][3*g:3*(g+1)]), wires=q)
    
    return [qml.expval(qml.Hermitian(density_matrix(state_labels[i]), wires=[i])) for i in range(n_qubits)]


In [265]:
class class_weights(tf.keras.layers.Layer):
    def __init__(self):
        super(class_weights, self).__init__()
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value=w_init(shape=(1, n_class), dtype="float32"),
            trainable=True,
        )

    def call(self, inputs):
        return (inputs * self.w)

In [266]:
tf.keras.backend.set_floatx('float32')
tf.keras.backend.floatx()

'float32'

In [267]:
X = tf.keras.Input(shape=(27,27,1), dtype=tf.float32)

conv_layer_1 = tf.keras.layers.Conv2D(filters=1, kernel_size=[3,3], strides=[2,2], name='Conv_Layer_1', activation='relu')(X)
batch_norm_1 = tf.keras.layers.BatchNormalization(name='BatchNorm_Layer_1')(conv_layer_1)
conv_layer_2 = tf.keras.layers.Conv2D(filters=1, kernel_size=[3,3], strides=[2,2], name='Conv_Layer_2', activation='relu')(batch_norm_1)
batch_norm_2 = tf.keras.layers.BatchNormalization(name='BatchNorm_Layer_2')(conv_layer_2)

max_pool_layer = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=None, name='Max_Pool_Layer')(batch_norm_2)

reshapor_layer = tf.keras.layers.Reshape((9,), name='Reshapor_Layer')(max_pool_layer)

qlayer = qml.qnn.KerasLayer(qcircuit, {"params": (2, 3, n_class, 9)}, output_dim=n_class, name='Quantum_Layer')(reshapor_layer)

class_weights_layer = class_weights()(qlayer)

model = tf.keras.Model(inputs=X, outputs=class_weights_layer, name='Conv DRC')

In [268]:
X_train = np.array(X_train, dtype=np.float32)

In [269]:
model(X_train[0:32])

<tf.Tensor: shape=(32, 4), dtype=float32, numpy=
array([[ 0.0348328 , -0.00351673, -0.01986631, -0.00482257],
       [ 0.03426396, -0.00407391, -0.01935173, -0.00421902],
       [ 0.03443728, -0.00500171, -0.01737502, -0.00546223],
       [ 0.03454512, -0.00526645, -0.0168935 , -0.0059641 ],
       [ 0.03366666, -0.00397958, -0.01732437, -0.00413516],
       [ 0.03417629, -0.0036099 , -0.01800663, -0.00448901],
       [ 0.03353693, -0.00429389, -0.01843508, -0.00601237],
       [ 0.03451999, -0.00449004, -0.02422719, -0.00574664],
       [ 0.03472823, -0.00453365, -0.01785051, -0.00545274],
       [ 0.03480954, -0.00448555, -0.01855109, -0.00440017],
       [ 0.0338163 , -0.00451793, -0.01916916, -0.00647768],
       [ 0.0335154 , -0.00330202, -0.01963958, -0.00446709],
       [ 0.03425787, -0.00458436, -0.01870888, -0.00548651],
       [ 0.03440836, -0.00433528, -0.01896425, -0.00464772],
       [ 0.0337249 , -0.00439615, -0.01823932, -0.00425534],
       [ 0.03354717, -0.00326849, -0

In [270]:
model.summary()

Model: "Conv DRC"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_35 (InputLayer)        [(None, 27, 27, 1)]       0         
_________________________________________________________________
Conv_Layer_1 (Conv2D)        (None, 13, 13, 1)         10        
_________________________________________________________________
BatchNorm_Layer_1 (BatchNorm (None, 13, 13, 1)         4         
_________________________________________________________________
Conv_Layer_2 (Conv2D)        (None, 6, 6, 1)           10        
_________________________________________________________________
BatchNorm_Layer_2 (BatchNorm (None, 6, 6, 1)           4         
_________________________________________________________________
Max_Pool_Layer (MaxPooling2D (None, 3, 3, 1)           0         
_________________________________________________________________
Reshapor_Layer (Reshape)     (None, 9)                 0  

In [271]:
import keras.backend as K

# def custom_loss(y_true, y_pred):
#     return K.sum(((y_true.shape[1]-2)*y_true+1)*K.square(y_true-y_pred))/len(y_true)

def custom_loss(y_true, y_pred):
    return K.sum(K.square(y_true-y_pred))/len(y_true)

In [272]:
opt = tf.keras.optimizers.Adam(learning_rate=0.1)
model.compile(opt, loss=custom_loss, metrics=["accuracy"])

In [273]:
model.fit(X_train, Y_train, epochs=20, batch_size=32, validation_data=(X_test, Y_test), verbose=1)

Epoch 1/20
25/25 [==============================] - 1016s 41s/step - loss: 0.7600 - accuracy: 0.3619 - val_loss: 0.7303 - val_accuracy: 0.5150
Epoch 2/20
25/25 [==============================] - 1016s 41s/step - loss: 0.5512 - accuracy: 0.6068 - val_loss: 1.1225 - val_accuracy: 0.2200
Epoch 3/20
25/25 [==============================] - 1013s 41s/step - loss: 0.4728 - accuracy: 0.7127 - val_loss: 1.2476 - val_accuracy: 0.1600
Epoch 4/20
25/25 [==============================] - 1011s 40s/step - loss: 0.3691 - accuracy: 0.7821 - val_loss: 1.2091 - val_accuracy: 0.2650
Epoch 5/20
25/25 [==============================] - 1011s 40s/step - loss: 0.3847 - accuracy: 0.7776 - val_loss: 0.9869 - val_accuracy: 0.4200
Epoch 6/20
25/25 [==============================] - 1011s 40s/step - loss: 0.3414 - accuracy: 0.8142 - val_loss: 1.1193 - val_accuracy: 0.4100
Epoch 7/20
25/25 [==============================] - 1012s 40s/step - loss: 0.3474 - accuracy: 0.7902 - val_loss: 0.7540 - val_accuracy: 0.3750

In [239]:
predict_test = model.predict(X_test)